In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.7/112.7 KB 15.0 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=1ae2df4363350936d36301ceee7b6116b7b5575b14e24fffccb62b1a5f55aab1
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip
  

In [1]:
import pandas as pd
import random as rd
import xgboost as xgb
import optuna
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from tqdm.notebook import tqdm_notebook
from pathlib import Path 

pd.set_option('display.max_columns', None)

### 1. Load the Data

In [2]:
def label_3_category(x) :
  if x==65 or x==66 :
    return 0
  elif x==67 or x==68:
    return 1
  elif x==69 or x==70 :
    return 2

def label_emotions(x) :
  if x==65 or x==67 or x==69:
    return 0
  elif x==66 or x==68 or x==70:
    return 1
def label_test(x) :
  if x==65:
    return 0
  elif x==66:
    return 1
def label_subj(x) :
  if x<=0:
    return 0
  elif x>=0.01:
    return 1

In [3]:
df = pd.read_csv("C:/Users\Bruno\Documents\Meta_Output\Without_AU_c\All_Subs_Diff_Modules_not_norm.csv", sep=',')
# df = df.drop(df[(df.Condition == 67) | (df.Condition == 68)].index, axis=0)
print(df)
df.drop(['pose1', '_30'], axis=1, inplace=True)
df.drop(['Emotion', 'Presence'], axis=1, inplace=True)
# df['label_3_category'] = df['Condition'].apply(lambda x : label_3_category(x))
# df['label_emotions'] = df['Condition'].apply(lambda x : label_emotions(x))
df['label_test'] = df['Condition'].apply(lambda x : label_test(x))
# df['label_subj'] = df['Presence'].apply(lambda x : label_subj(x))

       Subject  Condition         pose0   Emotion  Presence  pose1     pose2  \
0       CHE324       65.0  9.617000e-06  0.444444 -0.637778    0.0  0.000000   
1       CHE324       65.0  4.814770e-03  0.444444 -0.637778    0.0  0.004836   
2       CHE324       65.0  6.820000e-07  0.444444 -0.637778    0.0  0.000000   
3       CHE324       65.0  1.473792e-03  0.444444 -0.637778    0.0  0.001650   
4       CHE324       65.0  4.880052e-03  0.444444 -0.637778    0.0  0.000000   
...        ...        ...           ...       ...       ...    ...       ...   
662927  TAI310       66.0  7.610000e-06 -0.593333 -0.555556    0.0  0.000000   
662928  TAI310       66.0  2.663500e-05 -0.593333 -0.555556    0.0  0.000000   
662929  TAI310       66.0  3.043900e-05 -0.593333 -0.555556    0.0  0.000000   
662930  TAI310       66.0  2.282900e-05 -0.593333 -0.555556    0.0  0.000000   
662931  TAI310       66.0  2.283000e-05 -0.593333 -0.555556    0.0  0.000000   

           pose3     pose4     pose5   

In [4]:
def normalize_df (df):
  # For first patient
  df_normalized = df[df['Subject']=='CHE324']
  scaler = MinMaxScaler()
  df_normalized.iloc[:,2:-1] = scaler.fit_transform(df_normalized.iloc[:,2:-1])

  # For the rest
  subject_ids = list(df.Subject.unique())[1:]
  for subject in subject_ids :
    df_subject = df[df['Subject']==subject]
    scaler = MinMaxScaler()
    df_subject.iloc[:,2:-1] = scaler.fit_transform(df_subject.iloc[:,2:-1])
    df_normalized = df_normalized.append(df_subject)
  
  return df_normalized

In [5]:

df_normalized = normalize_df(df)

C:\Users\Bruno\AppData\Local\Temp\ipykernel_1288\4125737943.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normalized.iloc[:,2:-1] = scaler.fit_transform(df_normalized.iloc[:,2:-1])
C:\Users\Bruno\AppData\Local\Temp\ipykernel_1288\4125737943.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subject.iloc[:,2:-1] = scaler.fit_transform(df_subject.iloc[:,2:-1])
C:\Users\Bruno\AppData\Local\Temp\ipykernel_1288\4125737943.py:13: FutureWarning: The frame.append method is deprecated and will be remov

C:\Users\Bruno\AppData\Local\Temp\ipykernel_1288\4125737943.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_normalized = df_normalized.append(df_subject)
C:\Users\Bruno\AppData\Local\Temp\ipykernel_1288\4125737943.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subject.iloc[:,2:-1] = scaler.fit_transform(df_subject.iloc[:,2:-1])
C:\Users\Bruno\AppData\Local\Temp\ipykernel_1288\4125737943.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_normalized = df_normalized.append(df_subject)
C:\Users\Bruno\AppData\Local\Temp\ipykernel_1288\4125737943.py:12: Se

### 2. Preprocessing

In [6]:
def split_df (valid_ids, test_ids, df_normalized = df_normalized):
  print('splitting dataframe ...')
  df_train = df_normalized[~df_normalized['Subject'].isin(valid_ids+test_ids)]
  df_valid = df_normalized[df_normalized['Subject'].isin(valid_ids)]
  df_test = df_normalized[df_normalized['Subject'].isin(test_ids)]
  return df_train, df_valid, df_test

In [7]:
# df_train, df_valid, df_test = split_df()

### 3. Creation of the MA features and Split

In [8]:
def create_dataset (df_train, df_valid, df_test, win_size = 1000, labels = 'label_test'):
  print('creating MA dataset ...')
  # For train set:
  df_train_MA = df_train.groupby(by=['Condition', 'Subject']).rolling(win_size).mean() # moving average over 30 points

  df_train_MA.dropna(inplace=True) # drop the na rows created
  df_train_MA = df_train_MA.reset_index() # make the Condition as column
  df_train_MA.drop(columns=df_train_MA.columns[2], axis=1, inplace=True) # remove the column created with reset_index()

  X_train = df_train_MA.iloc[:, 2:-1]
  y_train = df_train_MA[labels]

  # For validation set:
  df_valid_MA = df_valid.groupby(by=['Condition', 'Subject']).rolling(win_size).mean() # moving average over 30 points

  df_valid_MA.dropna(inplace=True) # drop the na rows created
  df_valid_MA = df_valid_MA.reset_index() # make the Condition as column
  df_valid_MA.drop(columns=df_valid_MA.columns[2], axis=1, inplace=True) # remove the column created with reset_index()

  X_valid = df_valid_MA.iloc[:, 2:-1]
  y_valid = df_valid_MA[labels]

  # For test set :
  df_test_MA = df_test.groupby(by=['Condition', 'Subject']).rolling(win_size).mean() # moving average over 30 points

  df_test_MA.dropna(inplace=True) # drop the na rows created
  df_test_MA = df_test_MA.reset_index() # make the Condition as column
  df_test_MA.drop(columns=df_test_MA.columns[2], axis=1, inplace=True) # remove the column created with reset_index()

  X_test = df_test_MA.iloc[:, 2:-1]
  y_test = df_test_MA[labels]

  return X_train, y_train, X_valid, y_valid, X_test, y_test


In [9]:
# X_train, y_train, X_valid, y_valid, X_test, y_test = create_dataset(df_train, 1000, 'label_3_category')

### 4. Definiton and tunning the model

In [10]:
def objective(trial, X_train, y_train, X_valid, y_valid):
    
    param = {
        'max_depth': trial.suggest_int('max_depth', 5, 14),
        'n_estimators': trial.suggest_int('n_estimators', 1000, 15000, 100),
        'reg_alpha': trial.suggest_int('reg_alpha', 1, 50),
        'reg_lambda': trial.suggest_int('reg_lambda', 5, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1.0),
        }
   
    model = XGBClassifier(random_state=42,
                          objective='multi:softmax',
                          num_class = 2,
                          eval_metric='mlogloss',
                          use_label_encoder=False,
                          tree_method='gpu_hist',
                          **param ) 
    
    model.fit(X_train, y_train,eval_set=[(X_valid, y_valid)], early_stopping_rounds=150,verbose=False)
    preds = model.predict(X_valid)
    acc = accuracy_score(y_valid, preds)
    return acc

In [11]:
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=10)

### 5. Train

In [12]:
# model_optimal = XGBClassifier(random_state=42,
#                           num_class = 4,
#                           eval_metric='mlogloss',
#                           use_label_encoder=False,
#                           tree_method='gpu_hist',
#                           **study.best_params) 
# model_optimal.fit(X_train, y_train)

### 6. Results

In [13]:
# y_pred = model_optimal.predict(X_test)
# acc_test = accuracy_score(y_pred, y_test)
# print("The accuracy on the test set is :{:2f} .".format(acc_test))

In [14]:
def save_features (model, df, filename):
  features = list(df.columns.values[2:-1])
  feature_imp = pd.DataFrame(sorted(zip(model.feature_importances_, features)), columns=['Value','Feature'])
  #save to csv
  #feature_imp.to_csv("C:/Users/Bruno/Documents/Subjects_4_Preplotting/" + filename + "69vs70.csv")

In [15]:
# feature_imp = pd.DataFrame(sorted(zip(model_optimal.feature_importances_, features)), columns=['Value','Feature'])

# plt.figure(figsize=(18, 20))
# sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
# plt.title('XGboost Features (avg over folds)')
# plt.tight_layout()
# plt.show()

In [16]:
# model_optimal.save_model('gdrive/My Drive/PSAT/Notebooks/saved_models/MA60_99,98.json')

Cross Validation

In [17]:
def get_valid_train_ids (subject, subject_ids):

  print('get ids ...')
  valid_ids = []
  test_ids = []
  valid_ids.append(subject_ids[subject])
#   valid_ids.append(subject_ids[(subject + 1)%16])

  test_ids.append(subject_ids[(subject + 1)%16])
#   test_ids.append(subject_ids[(subject + 3)%15])

  return valid_ids, test_ids

In [18]:
def get_best_params (X_train, y_train, X_valid, y_valid, n_trials = 10):
  study = optuna.create_study(direction='maximize')

  objective_with_param = lambda trial: objective(trial, X_train, y_train, X_valid, y_valid)

  study.optimize(objective_with_param, n_trials=n_trials)

  return study.best_params

In [24]:
def cross_valid ():
    results = []
    subject_ids = list(df.Subject.unique())

    for sub in range(0, 16, 1):
        valid_ids, test_ids = get_valid_train_ids(sub, subject_ids)

        df_train, df_valid, df_test = split_df(valid_ids, test_ids)

        X_train, y_train, X_valid, y_valid, X_test, y_test = create_dataset(df_train, df_valid, df_test, 1000, labels='label_test')

        best_params = get_best_params (X_train, y_train, X_valid, y_valid)

        model_optimal = XGBClassifier(random_state=42,
                                objective='multi:softmax',
                                num_class = 2,
                                eval_metric='mlogloss',
                                use_label_encoder=False,
                                tree_method='gpu_hist',
                                **best_params) 
        print('training ...')
        model_optimal.fit(X_train, y_train)


        print('predicting ...')
        y_pred = model_optimal.predict(X_test)
        acc_test = accuracy_score(y_pred, y_test)

        valid_ids_str = '+'.join(map(str, valid_ids))
        test_ids_str = '+'.join(map(str, test_ids))

        print("The accuracy on " + test_ids_str + " is :{:2f} .".format(acc_test))
        #save the results with the list of subject (in the title)
        results.append((valid_ids_str + '_' + test_ids_str, acc_test))

        #save the feature importances
        print('saving ...')
        save_features(model_optimal, df_train, valid_ids_str + test_ids_str)
    
    print('cross validation finished')
    return results


In [25]:
# results = cross_valid()

In [26]:
# results

In [27]:
# results = np.array(results)
# mean = np.mean(results[:,1], dtype= np.float32)
# mean

RESULTS

In [29]:
results = cross_valid()

get ids ...
splitting dataframe ...
creating MA dataset ...


[I 2023-04-12 11:06:12,717] A new study created in memory with name: no-name-26ccc652-d987-4fc4-aff7-550d33dcdf85
c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-04-12 11:06:14,897] Trial 0 finished with value: 0.44235695986336465 and parameters: {'max_depth': 5, 'n_estimators': 7300, 'reg_alpha': 30, 'reg_lambda': 68, 'min_child_weight': 16, 'colsample_bytree': 0.7889843286361897}. Best is trial 0 with value: 0.44235695986336465.
c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sk

c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


training ...
predicting ...
The accuracy on DES316 is :0.597462 .
saving ...
get ids ...
splitting dataframe ...
creating MA dataset ...


[I 2023-04-12 11:06:58,434] A new study created in memory with name: no-name-3c490cc5-5f53-4bbf-b1a7-417aee3914c2
c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-04-12 11:07:01,604] Trial 0 finished with value: 0.7629396151293961 and parameters: {'max_depth': 14, 'n_estimators': 12100, 'reg_alpha': 1, 'reg_lambda': 41, 'min_child_weight': 6, 'colsample_bytree': 0.8481805029040056}. Best is trial 0 with value: 0.7629396151293961.
c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\skle

...

c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


training ...
predicting ...
The accuracy on TAI310 is :0.676265 .
saving ...
get ids ...
splitting dataframe ...
creating MA dataset ...


[I 2023-04-12 11:20:10,409] A new study created in memory with name: no-name-253aa643-4743-4987-8696-c3a80e53d481
c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2023-04-12 11:20:12,944] Trial 0 finished with value: 0.7675153793574846 and parameters: {'max_depth': 14, 'n_estimators': 6100, 'reg_alpha': 38, 'reg_lambda': 44, 'min_child_weight': 12, 'colsample_bytree': 0.4829461524127532}. Best is trial 0 with value: 0.7675153793574846.
c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\skl

c:\users\bruno\documents\pycharmprojects\randomforest\venv\lib\site-packages\xgboost\sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


training ...
predicting ...
The accuracy on CHE324 is :0.447993 .
saving ...
cross validation finished


In [30]:
results = np.array(results)
print(results)
mean = np.mean(results[:,1], dtype= np.float32)
df = pd.DataFrame(results, columns = ['Fold','Accuracy'])
# df.to_csv('C:/Users\Bruno\Documents\RF_LSTM\Resultats_Accuracy/65vs66.csv', sep=',', index=False)
print(mean)

[['CHE324_DES316' '0.5974618447246185']
 ['DES316_EDC312' '0.563548331315926']
 ['EDC312_GIC326' '0.8232415394824154']
 ['GIC326_GRL315' '0.7844366432631227']
 ['GRL315_HAA328' '0.5798855816267308']
 ['HAA328_MAL313' '0.5184067085953878']
 ['MAL313_MIL332' '0.8113224315677213']
 ['MIL332_MOP309' '0.6262984764542936']
 ['MOP309_PAP319' '0.7046680148297944']
 ['PAP319_POA318' '0.8350836888743026']
 ['POA318_PRP307' '0.4767269736842105']
 ['PRP307_REL317' '0.964509394572025']
 ['REL317_ROL311' '0.5025060490839959']
 ['ROL311_SAE308' '0.44438610308917476']
 ['SAE308_TAI310' '0.6762645249487355']
 ['TAI310_CHE324' '0.4479931682322801']]
0.6472962


FEATURE IMPORTANCE

In [110]:
features_importance = pd.DataFrame()
for filename in os.listdir("C:/Users/Bruno/Documents/Subjects_4_Preplotting/"):
    if filename.endswith('csv'):
        data = pd.read_csv("C:/Users/Bruno/Documents/Subjects_4_Preplotting/" + filename, sep=',')
        features_importance = features_importance.append(data, ignore_index=True)
features_importance = features_importance.drop(columns=features_importance.columns[0])

feat_mean = features_importance.groupby('Feature', as_index=False).mean()

plt.figure(figsize=(18, 20))
sns.barplot(data=feat_mean.sort_values(by="Value", ascending=False), x='Value', y='Feature')
plt.title("Moyenne de l'importance des features pour la classification après cross validation 14-1-1")
plt.tight_layout()
plt.show()

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'C:/Users/Bruno/Documents/Subjects_4_Preplotting/'

## ----END---- ##